In [1]:
!pip install onnx onnxruntime

In [2]:
import math
import onnx
import onnxruntime as ort
import numpy as np
from onnx import TensorProto

# Define input and output tensor names
input1_name = "X"
maxpool_output_name1 = "Y"
maxpool_output_name2 = "Indices"


# Create the ONNX model with maxpool operator
def create_maxpool_model(input_shape, output_shape, dtype):

    #Create "input-rank" input tensor
    input1 = onnx.helper.make_tensor_value_info(input1_name, dtype, input_shape)

    # Create output tensor (final result after maxpool operation)
    maxpool_output1 = onnx.helper.make_tensor_value_info(maxpool_output_name1, dtype, output_shape)
    maxpool_output2 = onnx.helper.make_tensor_value_info(maxpool_output_name2, TensorProto.INT64, output_shape)

    # Define clip node
    maxpool_node = onnx.helper.make_node(
        "MaxPool",
        inputs=[input1_name],
        outputs=[maxpool_output_name1, maxpool_output_name2],
        ceil_mode = 0,
        dilations=[1,1],
        kernel_shape=[3,3],
        pads=[0, 0, 0, 0],
        strides=[1, 1],
        auto_pad='NOTSET',
    )

    # Create the ONNX graph
    graph_def = onnx.helper.make_graph(
        [maxpool_node],
        "Test_MaxPool",
        [input1],
        [maxpool_output1, maxpool_output2],
    )

    # Create the ONNX model
    model = onnx.helper.make_model(graph_def, opset_imports=[onnx.helper.make_opsetid("", 22)]) # Explicitly set opset to 13
    model.ir_version = 10 
    onnx.checker.check_model(model)

    # Save the model
    onnx.save(model, "maxpool.onnx")

    # Load and run the model with ONNX Runtime
    session = ort.InferenceSession("maxpool.onnx")
    return session

def do_maxpool(x, session):
    # Run inference
    output = session.run(None, {input1_name: x})

    x_f = (np.array2string(x, separator=',', max_line_width=np.inf).replace('\n', '\n'))
    y_f = (np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', '\n'))
    indices_f = (np.array2string(output[1], separator=',', max_line_width=np.inf).replace('\n', '\n'))

    # Display results
    print(f"X = \n{x_f}")
    print(f"Y = \n{y_f}")
    print(f"Indices = \n{indices_f}")

def compute_MaxPool_output_shape(input_shape, kernel_shape, pads, dilation, strides):

    dx0, dx1, dx2, dx3 = input_shape
    dw0, dw1 = kernel_shape


    myalpha = dx2 + pads[0] + pads[2]
    mybeta = dx3 + pads[1] + pads[3]
    mytheta = (dilation[0] * (dw0 - 1)) + 1
    mygamma = (dilation[1] * (dw1 - 1)) + 1
#   dY_2 = \left\lfloor{(dX_2 + pad\_shape[0] - dilations[0] * (kernel\_shape[0] - 1) - 1) / (strides[0] + 1)}\right\rfloor
    mydy2 = math.floor(((myalpha - (mytheta)) / strides[0]) + 1)
#   dY_3 = \left\lfloor{(dX_3 + pad\_shape[1] - dilations[1] * (kernel\_shape[1] - 1) - 1) / (strides[1] + 1)} \right\rfloor
    mydy3 = math.floor(((mybeta - (mygamma)) / strides[1]) + 1)
    
    output_shape = [dx0, dx1, mydy2, mydy3]

    return output_shape


def pad_4d_spatial_js(tensor, paddings):
    """
    Pads the last two dimensions (H, W) of a 4D tensor.
    paddings = (top, left, bottom, right)
    """
    N, C, H, W = tensor.shape
    top, left, bottom, right = paddings
    
    # 1. Calculate the new spatial dimensions
    new_H = H + top + bottom
    new_W = W + left + right
    
    # 2. Create the new 4D "canvas" of zeros
    # The Batch (N) and Channel (C) dimensions remain the same
#    padded_tensor = np.zeros((N, C, new_H, new_W), dtype=tensor.dtype)
    padded_tensor = np.full((N, C, new_H, new_W), -np.inf)
    
    # 3. Insert the original tensor into the padded one
    # We use ":" for N and C to select all batches and channels
    # We use slicing for the H and W dimensions
    padded_tensor[:, :, top : top + H, left : left + W] = tensor
    
    return padded_tensor

def extract_max_dilated_from_dilated_window(X_p, b, c, m, n, strides, dilations, dW):
    """
    Extracts the maximum value from a dilated window.
    
    Args:
        X_p: The 2D padded input array.
        m, n: Integer constants representing the current output position.
        strides: Tuple (s_h, s_w) for step size.
        dilations: Tuple (d_h, d_w) for spacing between window elements.
        dW: Tuple (dW0, dW1) for the window height and width.
    """
    dW0, dW1 = dW
    s_h, s_w = strides
    d_h, d_w = dilations
   
    # Calculate the base (top-left) coordinates based on the stride
    base_h = m * s_h
    base_w = n * s_w
    
    # Initialize max_val with the first element of the window (h=0, w=0)
    max_val = X_p[b, c, base_h, base_w]
    
    # Iterate through the window defined by dW0 and dW1
    for h in range(dW0):
        for w in range(dW1):
            # Calculate the current dilated coordinates
            curr_h = base_h + h * d_h
            curr_w = base_w + w * d_w
            
            # Extract the value at these coordinates
            val = X_p[b, c, curr_h, curr_w]
            
            # Update the maximum found so far
            if val > max_val:
                max_val = val
                
    return max_val

def MaxPool(X, kernel_shape, strides, dilation, pads):

    Y_dims = compute_MaxPool_output_shape(X.shape, kernel_shape, pads, dilation, strides)

    Y = np.zeros(Y_dims)

    X_p = pad_4d_spatial_js(X, pads)
    dX_p0, dX_p1, dX_p2, dX_p3 = X_p.shape
    dY_p0, dY_p1, dY_p2, dY_p3 = Y.shape

    xp_f = (np.array2string(X_p, separator=',', max_line_width=np.inf).replace('\n', '\n'))
    print(f"MaxPool: X_p = \n{xp_f}")

    for b in range(dX_p0):
       for c in range(dX_p1): 
         for m in range(dY_p2):
           for n in range(dY_p3):
             Y[b, c, m, n] = extract_max_dilated_from_dilated_window(X_p, b, c, m, n, strides, dilation, kernel_shape)

    Y_f = (np.array2string(Y, separator=',', max_line_width=np.inf).replace('\n', '\n'))
    print(f"MaxPool: Y = \n{Y_f}")


np.set_printoptions(precision=None, floatmode='fixed')

## Nominal Cases

In [3]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.DOUBLE
x = np.random.normal(3, 2.5, size=(1, 1, 8, 8))
session = create_maxpool_model(x.shape, [1, 1, 6, 6], onnx_type)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], [0,0,0,0])

X = 
[[[[ 4.82722798, 2.03801341, 3.80848570, 6.35570142,-0.55493686, 4.63352680, 1.10544092, 4.35162354],
   [ 5.06084425, 2.19555868, 1.96680152, 3.34915036, 2.93807758,-0.29482237, 4.89450566, 3.40603468],
   [ 7.96935843, 6.94398320, 6.55524589, 3.17207158, 0.36220111, 4.60131929, 0.94936148, 4.78565390],
   [ 1.12453129, 0.07466255, 1.58861500, 8.90349586, 4.29221765, 2.63331320, 0.18132976, 0.30256856],
   [-0.43043683, 8.13761571, 2.40942047, 3.29001674,10.16470362, 0.93903026, 4.34205509, 4.93459564],
   [ 2.25575465, 2.45549916, 3.52898119,-0.15229401, 3.55693720, 8.57267025, 2.33238574, 2.86368324],
   [ 2.60264036, 0.75261116, 1.89291850, 5.22739202, 2.87911426, 1.83295584,-0.83814927, 5.22659999],
   [ 3.63450877,-1.54396494, 2.15440818,-2.80311836, 2.57784278, 2.83220123, 8.86424586,-2.04131139]]]]
Y = 
[[[[ 7.96935843, 6.94398320, 6.55524589, 6.35570142, 4.89450566, 4.89450566],
   [ 7.96935843, 8.90349586, 8.90349586, 8.90349586, 4.89450566, 4.89450566],
   [ 8.13761571,

## No nominal cases (nan and inf values)

In [4]:
# Case NN1:BLA BLA BLA
onnx_type = onnx.TensorProto.FLOAT
data = [[[[ np.nan, 0.39876820, 4.37934912, 3.15053735, 2.68933629, 2.04979010, 2.22631180, 2.91909200],
   [ 5.17326681, 1.48754092, 3.88272884, 4.88280180, 3.82933887, 6.27754705, 4.06093024, 8.06596070],
   [ 3.78974562, 6.38371591,-2.17753597,-3.33859408, 1.70501236, 3.22134587,-4.71752299, 1.27690398],
   [ 5.15344877, 0.95474975, 3.61153436, 1.37445399, 2.46553948,-0.53501892, 4.85525366, 5.08866624],
   [ 3.31550533, 6.35535590, 3.82945834, 5.74075666, 7.07525101, 5.12485980, 5.06112666, 1.61803044],
   [ 5.09879424, 4.01784619, 3.13594030, 4.23551037, 2.28690697, 3.19434453, 2.55583487, 1.47128808],
   [ 3.92282805,-0.15807763, 2.35843771, 4.97382280, 2.43473158, 1.18582722, 5.70581444, 0.53569289],
   [ 0.42778469, 2.58801158, 3.42065351, 4.70146247,-1.65992634, 1.79832601, 1.84941921, 1.67840883]]]]
x = np.array(data, dtype=np.float32)
session = create_maxpool_model(x.shape, [1, 1, 6, 6], onnx_type)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], [0,0,0,0])

X = 
[[[[        nan, 0.39876819, 4.37934923, 3.15053725, 2.68933630, 2.04979014, 2.22631168, 2.91909194],
   [ 5.17326689, 1.48754096, 3.88272882, 4.88280201, 3.82933879, 6.27754688, 4.06093025, 8.06596088],
   [ 3.78974557, 6.38371611,-2.17753601,-3.33859420, 1.70501232, 3.22134590,-4.71752310, 1.27690399],
   [ 5.15344858, 0.95474976, 3.61153436, 1.37445402, 2.46553946,-0.53501892, 4.85525370, 5.08866644],
   [ 3.31550527, 6.35535574, 3.82945824, 5.74075651, 7.07525110, 5.12485981, 5.06112671, 1.61803043],
   [ 5.09879446, 4.01784611, 3.13594031, 4.23551035, 2.28690696, 3.19434452, 2.55583477, 1.47128808],
   [ 3.92282796,-0.15807763, 2.35843778, 4.97382259, 2.43473148, 1.18582726, 5.70581436, 0.53569287],
   [ 0.42778468, 2.58801150, 3.42065358, 4.70146227,-1.65992630, 1.79832602, 1.84941924, 1.67840886]]]]
Y = 
[[[[6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.06596088],
   [6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.06596088],
   [6.38371611,6.38371611,7.

In [5]:
# Case NN1:BLA BLA BLA
onnx_type = onnx.TensorProto.FLOAT
data = [[[[ 3.15168382, 0.39876820, 4.37934912, 3.15053735, 2.68933629, 2.04979010, 2.22631180, 2.91909200],
   [ 5.17326681, 1.48754092, 3.88272884, 4.88280180, 3.82933887, 6.27754705, 4.06093024, 8.06596070],
   [ 3.78974562, 6.38371591,-2.17753597,-3.33859408, 1.70501236, 3.22134587,-4.71752299, 1.27690398],
   [ 5.15344877, 0.95474975, 3.61153436, 1.37445399, 2.46553948,-0.53501892, 4.85525366, 5.08866624],
   [ 3.31550533, 6.35535590, 3.82945834, 5.74075666, 7.07525101, 5.12485980, 5.06112666, 1.61803044],
   [ 5.09879424, 4.01784619, 3.13594030, 4.23551037, 2.28690697, 3.19434453, 2.55583487, 1.47128808],
   [ 3.92282805,-0.15807763, 2.35843771, 4.97382280, 2.43473158, 1.18582722, 5.70581444, 0.53569289],
   [ 0.42778469, 2.58801158, 3.42065351, 4.70146247,-1.65992634, 1.79832601, 1.84941921, np.nan]]]]
x = np.array(data, dtype=np.float32)
session = create_maxpool_model(x.shape, [1, 1, 6, 6], onnx_type)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], [0,0,0,0])

X = 
[[[[ 3.15168381, 0.39876819, 4.37934923, 3.15053725, 2.68933630, 2.04979014, 2.22631168, 2.91909194],
   [ 5.17326689, 1.48754096, 3.88272882, 4.88280201, 3.82933879, 6.27754688, 4.06093025, 8.06596088],
   [ 3.78974557, 6.38371611,-2.17753601,-3.33859420, 1.70501232, 3.22134590,-4.71752310, 1.27690399],
   [ 5.15344858, 0.95474976, 3.61153436, 1.37445402, 2.46553946,-0.53501892, 4.85525370, 5.08866644],
   [ 3.31550527, 6.35535574, 3.82945824, 5.74075651, 7.07525110, 5.12485981, 5.06112671, 1.61803043],
   [ 5.09879446, 4.01784611, 3.13594031, 4.23551035, 2.28690696, 3.19434452, 2.55583477, 1.47128808],
   [ 3.92282796,-0.15807763, 2.35843778, 4.97382259, 2.43473148, 1.18582726, 5.70581436, 0.53569287],
   [ 0.42778468, 2.58801150, 3.42065358, 4.70146227,-1.65992630, 1.79832602, 1.84941924,        nan]]]]
Y = 
[[[[6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.06596088],
   [6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.06596088],
   [6.38371611,6.38371611,7.

In [6]:
# Case NN1:BLA BLA BLA
onnx_type = onnx.TensorProto.FLOAT
data = [[[[ np.nan, 0.39876820, 4.37934912, 3.15053735, 2.68933629, 2.04979010, 2.22631180, 2.91909200],
   [ 5.17326681, 1.48754092, 3.88272884, 4.88280180, 3.82933887, 6.27754705, 4.06093024, 8.06596070],
   [ 3.78974562, 6.38371591,-2.17753597,-3.33859408, 1.70501236, 3.22134587,-4.71752299, 1.27690398],
   [ 5.15344877, 0.95474975, 3.61153436, 1.37445399, np.nan,-0.53501892, 4.85525366, 5.08866624],
   [ 3.31550533, 6.35535590, 3.82945834, 5.74075666, 7.07525101, 5.12485980, 5.06112666, 1.61803044],
   [ 5.09879424, 4.01784619, 3.13594030, 4.23551037, 2.28690697, 3.19434453, 2.55583487, 1.47128808],
   [ 3.92282805,-0.15807763, 2.35843771, 4.97382280, 2.43473158, 1.18582722, 5.70581444, 0.53569289],
   [ 0.42778469, 2.58801158, 3.42065351, 4.70146247,-1.65992634, 1.79832601, 1.84941921, np.nan]]]]
x = np.array(data, dtype=np.float32)
session = create_maxpool_model(x.shape, [1, 1, 6, 6], onnx_type)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], [0,0,0,0])

X = 
[[[[        nan, 0.39876819, 4.37934923, 3.15053725, 2.68933630, 2.04979014, 2.22631168, 2.91909194],
   [ 5.17326689, 1.48754096, 3.88272882, 4.88280201, 3.82933879, 6.27754688, 4.06093025, 8.06596088],
   [ 3.78974557, 6.38371611,-2.17753601,-3.33859420, 1.70501232, 3.22134590,-4.71752310, 1.27690399],
   [ 5.15344858, 0.95474976, 3.61153436, 1.37445402,        nan,-0.53501892, 4.85525370, 5.08866644],
   [ 3.31550527, 6.35535574, 3.82945824, 5.74075651, 7.07525110, 5.12485981, 5.06112671, 1.61803043],
   [ 5.09879446, 4.01784611, 3.13594031, 4.23551035, 2.28690696, 3.19434452, 2.55583477, 1.47128808],
   [ 3.92282796,-0.15807763, 2.35843778, 4.97382259, 2.43473148, 1.18582726, 5.70581436, 0.53569287],
   [ 0.42778468, 2.58801150, 3.42065358, 4.70146227,-1.65992630, 1.79832602, 1.84941924,        nan]]]]
Y = 
[[[[6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.06596088],
   [6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.06596088],
   [6.38371611,6.38371611,7.

In [7]:
# Case NN2:BLA BLA BLA
onnx_type = onnx.TensorProto.FLOAT
data = [[[[ 3.15168382, 0.39876820, 4.37934912, 3.15053735, 2.68933629, 2.04979010, 2.22631180, 2.91909200],
   [ 5.17326681, 1.48754092, 3.88272884, 4.88280180, 3.82933887, 6.27754705, 4.06093024, 8.06596070],
   [ 3.78974562, 6.38371591,-2.17753597,-3.33859408, 1.70501236, 3.22134587,-4.71752299, 1.27690398],
   [ 5.15344877, 0.95474975, 3.61153436, 1.37445399, 2.46553948,-0.53501892, 4.85525366, 5.08866624],
   [ 3.31550533, 6.35535590, 3.82945834, 5.74075666, 7.07525101, 5.12485980, 5.06112666, 1.61803044],
   [ 5.09879424, 4.01784619, 3.13594030, 4.23551037, 2.28690697, 3.19434453, 2.55583487, 1.47128808],
   [ 3.92282805,-0.15807763, 2.35843771, 4.97382280, 2.43473158, 1.18582722, 5.70581444, 0.53569289],
   [ 0.42778469, 2.58801158, 3.42065351, 4.70146247,-1.65992634, 1.79832601, 1.84941921, np.inf]]]]
x = np.array(data, dtype=np.float32)
session = create_maxpool_model(x.shape, [1, 1, 6, 6], onnx_type)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], [0,0,0,0])

X = 
[[[[ 3.15168381, 0.39876819, 4.37934923, 3.15053725, 2.68933630, 2.04979014, 2.22631168, 2.91909194],
   [ 5.17326689, 1.48754096, 3.88272882, 4.88280201, 3.82933879, 6.27754688, 4.06093025, 8.06596088],
   [ 3.78974557, 6.38371611,-2.17753601,-3.33859420, 1.70501232, 3.22134590,-4.71752310, 1.27690399],
   [ 5.15344858, 0.95474976, 3.61153436, 1.37445402, 2.46553946,-0.53501892, 4.85525370, 5.08866644],
   [ 3.31550527, 6.35535574, 3.82945824, 5.74075651, 7.07525110, 5.12485981, 5.06112671, 1.61803043],
   [ 5.09879446, 4.01784611, 3.13594031, 4.23551035, 2.28690696, 3.19434452, 2.55583477, 1.47128808],
   [ 3.92282796,-0.15807763, 2.35843778, 4.97382259, 2.43473148, 1.18582726, 5.70581436, 0.53569287],
   [ 0.42778468, 2.58801150, 3.42065358, 4.70146227,-1.65992630, 1.79832602, 1.84941924,        inf]]]]
Y = 
[[[[6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.06596088],
   [6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.06596088],
   [6.38371611,6.38371611,7.

In [8]:
# Case NN3:BLA BLA BLA
onnx_type = onnx.TensorProto.FLOAT
data = [[[[ 3.15168382, 0.39876820, 4.37934912, 3.15053735, 2.68933629, 2.04979010, 2.22631180, 2.91909200],
   [ 5.17326681, 1.48754092, 3.88272884, 4.88280180, 3.82933887, 6.27754705, 4.06093024, 8.06596070],
   [ 3.78974562, 6.38371591,-2.17753597,-3.33859408, 1.70501236, 3.22134587,-4.71752299, 1.27690398],
   [ 5.15344877, 0.95474975, 3.61153436, 1.37445399, 2.46553948,-0.53501892, 4.85525366, 5.08866624],
   [ 3.31550533, 6.35535590, 3.82945834, 5.74075666, 7.07525101, 5.12485980, 5.06112666, 1.61803044],
   [ 5.09879424, 4.01784619, 3.13594030, 4.23551037, 2.28690697, 3.19434453, 2.55583487, 1.47128808],
   [ 3.92282805,-0.15807763, 2.35843771, 4.97382280, 2.43473158, 1.18582722, 5.70581444, 0.53569289],
   [ 0.42778469, 2.58801158, 3.42065351, 4.70146247,-1.65992634, 1.79832601, 1.84941921, -np.inf]]]]
x = np.array(data, dtype=np.float32)
session = create_maxpool_model(x.shape, [1, 1, 6, 6], onnx_type)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], [0,0,0,0])

X = 
[[[[ 3.15168381, 0.39876819, 4.37934923, 3.15053725, 2.68933630, 2.04979014, 2.22631168, 2.91909194],
   [ 5.17326689, 1.48754096, 3.88272882, 4.88280201, 3.82933879, 6.27754688, 4.06093025, 8.06596088],
   [ 3.78974557, 6.38371611,-2.17753601,-3.33859420, 1.70501232, 3.22134590,-4.71752310, 1.27690399],
   [ 5.15344858, 0.95474976, 3.61153436, 1.37445402, 2.46553946,-0.53501892, 4.85525370, 5.08866644],
   [ 3.31550527, 6.35535574, 3.82945824, 5.74075651, 7.07525110, 5.12485981, 5.06112671, 1.61803043],
   [ 5.09879446, 4.01784611, 3.13594031, 4.23551035, 2.28690696, 3.19434452, 2.55583477, 1.47128808],
   [ 3.92282796,-0.15807763, 2.35843778, 4.97382259, 2.43473148, 1.18582726, 5.70581436, 0.53569287],
   [ 0.42778468, 2.58801150, 3.42065358, 4.70146227,-1.65992630, 1.79832602, 1.84941924,       -inf]]]]
Y = 
[[[[6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.06596088],
   [6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.06596088],
   [6.38371611,6.38371611,7.

In [9]:
# Case NN4:BLA BLA BLA
onnx_type = onnx.TensorProto.FLOAT
data = [[[[ 3.15168382, 0.39876820, 4.37934912, 3.15053735, 2.68933629, 2.04979010, 2.22631180, 2.91909200],
   [ 5.17326681, 1.48754092, 3.88272884, 4.88280180, 3.82933887, 6.27754705, 4.06093024, 8.06596070],
   [ 3.78974562, 6.38371591,-2.17753597,-3.33859408, 1.70501236, 3.22134587,-4.71752299, 1.27690398],
   [ 5.15344877, 0.95474975, 3.61153436, 1.37445399, 2.46553948,-0.53501892, 4.85525366, 5.08866624],
   [ 3.31550533, 6.35535590, 3.82945834, 5.74075666, 7.07525101, 5.12485980, 5.06112666, 1.61803044],
   [ 5.09879424, 4.01784619, 3.13594030, 4.23551037, 2.28690697, 3.19434453, 2.55583487, 1.47128808],
   [ 3.92282805,-0.15807763, 2.35843771, 4.97382280, 2.43473158, 1.18582722,  np.nan, 0.53569289],
   [ 0.42778469, 2.58801158, 3.42065351, 4.70146247,-1.65992634, 1.79832601, 1.84941921, -np.inf]]]]
x = np.array(data, dtype=np.float32)
session = create_maxpool_model(x.shape, [1, 1, 6, 6], onnx_type)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], [0,0,0,0])

X = 
[[[[ 3.15168381, 0.39876819, 4.37934923, 3.15053725, 2.68933630, 2.04979014, 2.22631168, 2.91909194],
   [ 5.17326689, 1.48754096, 3.88272882, 4.88280201, 3.82933879, 6.27754688, 4.06093025, 8.06596088],
   [ 3.78974557, 6.38371611,-2.17753601,-3.33859420, 1.70501232, 3.22134590,-4.71752310, 1.27690399],
   [ 5.15344858, 0.95474976, 3.61153436, 1.37445402, 2.46553946,-0.53501892, 4.85525370, 5.08866644],
   [ 3.31550527, 6.35535574, 3.82945824, 5.74075651, 7.07525110, 5.12485981, 5.06112671, 1.61803043],
   [ 5.09879446, 4.01784611, 3.13594031, 4.23551035, 2.28690696, 3.19434452, 2.55583477, 1.47128808],
   [ 3.92282796,-0.15807763, 2.35843778, 4.97382259, 2.43473148, 1.18582726,        nan, 0.53569287],
   [ 0.42778468, 2.58801150, 3.42065358, 4.70146227,-1.65992630, 1.79832602, 1.84941924,       -inf]]]]
Y = 
[[[[6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.06596088],
   [6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.06596088],
   [6.38371611,6.38371611,7.

In [10]:
# Case NN4:BLA BLA BLA
onnx_type = onnx.TensorProto.FLOAT
data = [[[[ 3.15168382, 0.39876820, 4.37934912, 3.15053735, 2.68933629, 2.04979010, 2.22631180, 2.91909200],
   [ 5.17326681, 1.48754092, 3.88272884, 4.88280180, 3.82933887, 6.27754705, 4.06093024, 8.06596070],
   [ 3.78974562, 6.38371591,-2.17753597,-3.33859408, 1.70501236, 3.22134587,-4.71752299, 1.27690398],
   [ 5.15344877, 0.95474975, 3.61153436, 1.37445399, 2.46553948,-0.53501892, 4.85525366, 5.08866624],
   [ 3.31550533, 6.35535590, 3.82945834, 5.74075666, 7.07525101, 5.12485980, 5.06112666, 1.61803044],
   [ 5.09879424, 4.01784619, 3.13594030, 4.23551037, 2.28690697, 3.19434453, 2.55583487, 1.47128808],
   [ 3.92282805,-0.15807763, 2.35843771, 4.97382280, 2.43473158, 1.18582722,  np.nan, 0.53569289],
   [ 0.42778469, 2.58801158, 3.42065351, 4.70146247,-1.65992634, 1.79832601, 1.84941921, np.inf]]]]
x = np.array(data, dtype=np.float32)
session = create_maxpool_model(x.shape, [1, 1, 6, 6], onnx_type)
do_maxpool(x, session)
MaxPool(x, [3,3], [1,1], [1,1], [0,0,0,0])

X = 
[[[[ 3.15168381, 0.39876819, 4.37934923, 3.15053725, 2.68933630, 2.04979014, 2.22631168, 2.91909194],
   [ 5.17326689, 1.48754096, 3.88272882, 4.88280201, 3.82933879, 6.27754688, 4.06093025, 8.06596088],
   [ 3.78974557, 6.38371611,-2.17753601,-3.33859420, 1.70501232, 3.22134590,-4.71752310, 1.27690399],
   [ 5.15344858, 0.95474976, 3.61153436, 1.37445402, 2.46553946,-0.53501892, 4.85525370, 5.08866644],
   [ 3.31550527, 6.35535574, 3.82945824, 5.74075651, 7.07525110, 5.12485981, 5.06112671, 1.61803043],
   [ 5.09879446, 4.01784611, 3.13594031, 4.23551035, 2.28690696, 3.19434452, 2.55583477, 1.47128808],
   [ 3.92282796,-0.15807763, 2.35843778, 4.97382259, 2.43473148, 1.18582726,        nan, 0.53569287],
   [ 0.42778468, 2.58801150, 3.42065358, 4.70146227,-1.65992630, 1.79832602, 1.84941924,        inf]]]]
Y = 
[[[[6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.06596088],
   [6.38371611,6.38371611,4.88280201,6.27754688,6.27754688,8.06596088],
   [6.38371611,6.38371611,7.